# Interactively Search CMR for  ABoVE Collections & Granules

https://above.nasa.gov/implementation_plan/standard_projection.html#reference

`![ABoVE Grid AB](https://above.nasa.gov/images/standard_projection_image_20170621.png)`

In [1]:
from ABoVE import *

In [8]:
def updateme(*args, **kwargs):
    """ """
    print(*args)
    print(**kwargs)

app.tab.on(handler=updateme, names="value")

In [12]:
dir(granules)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_base_url',
 '_build_url',
 '_format',
 '_route',
 '_valid_formats_regex',
 '_valid_state',
 'bounding_box',
 'cloud_cover',
 'day_night_flag',
 'downloadable',
 'entry_title',
 'format',
 'get',
 'get_all',
 'granule_ur',
 'hits',
 'instrument',
 'line',
 'mode',
 'online_only',
 'options',
 'orbit_number',
 'parameters',
 'params',
 'platform',
 'point',
 'polygon',
 'short_name',
 'temporal',
 'version']

In [ ]:
for index, row in app.union.iterrows():
    print(index)
    print(row["boxes"])

In [ ]:
above_results_df

In [ ]:
app.selected_collections.iloc[0]

In [ ]:
for i,r in app.selected_collections.items():
    print(i["datasse"])
    print(r)


In [ ]:
for i, r in app.selected_collections.iterrows():
    print(i)
    print(r)

In [ ]:
#above_datasets = collections.keyword("*ABoVE*").get_all()

#print(len(above_datasets))
print(json.dumps(above_datasets[0], indent=4))

#for d in above_datasets:
#    print(d["dataset_id"])

In [ ]:
app.map.layers[2].layers[0].clear_layers()

In [ ]:
app.union.to_wkt()

In [ ]:
x,y = 

In [ ]:
y

In [ ]:
dir(app.union.exterior.coords)

In [ ]:
from CMR import CollectionQuery, GranuleQuery

collections = CollectionQuery()
granules = GranuleQuery()

In [ ]:
ornlabovedf = query_ornl_projects()
ornlabovedf.head()

In [ ]:
boxes = ornlabovedf.boxes
boxes[:5]

In [ ]:




#text_box = 
#text_box

In [ ]:


shape(test_geojson)
    
    
    
    
   def CMR_box_to_geojson(box, id=0):
    """ """
    
    box = box.split(" ")
    
    ll = [box[1], box[0]]
    lr = [box[3], box[0]]
    ur = [box[3], box[2]]
    ul = [box[1], box[2]]

    polygon = {"type": "Polygon", "coordinates": [ll, lr, ur,ul, ll]}

    return(polygon) 
    

In [ ]:
def geojson_intersect_cmr_box(selection_geojson, cmr_box):
    """ """
    
    selection_geom = shape(selection_geojson)
    cmr_geom = shape(cmr_box)

In [ ]:
print("Number of ABoVE datasets at ORNL DAAC:"+str(len(above)))
print(print(json.dumps(above[0], indent=4)))